In [123]:
import numpy as np # linear algebra 
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import accuracy_score, classification_report
import torch
import matplotlib.pyplot as plt
import os

In [124]:
dataset_name = "2D_Helm/"

#Configurpave to be dynamically adjusted
download_path = "../data/" #In the .gitignore list an

#n the rest of the code.
path_to_datasets = download_path + "/" + dataset_name 

if not os.path.exists(path_to_datasets):
    os.mkdir(path_to_datasets)

In [125]:
# Analytical solution for Grad(p).n = 0
# p(x,y) = cos(k*x) + cos(k*y)
# k = n*pi, n=1,2,3,4,5,....
# Watchout at least six nodes per wave

In [126]:
kappa = 8*np.pi
len_x = 5
len_y = 5

In [127]:
def target_func(x,y, kappa): 
    pres = -y*y + x*x
    return pres

TRAIN Data

In [128]:
x = torch.linspace(0,1,len_x, requires_grad=True)
y = torch.linspace(0,1,len_y, requires_grad=True)

In [129]:
X, Y = torch.meshgrid(x,y, indexing='ij')
f = X**2 + Y**2

In [130]:
f.shape

torch.Size([5, 5])

In [131]:
grad_fx, grad_fy = torch.autograd.grad(outputs=f, inputs=(X,Y), grad_outputs=torch.ones_like(f), create_graph=True)
# Compute second derivative
laplace_x = torch.autograd.grad(grad_fx, inputs=X, grad_outputs=torch.ones_like(grad_fx), create_graph=True)[0]
laplace_y = torch.autograd.grad(grad_fy, inputs=Y, grad_outputs=torch.ones_like(grad_fy), create_graph=True)[0]


In [132]:
laplace_y[0].shape

torch.Size([5])

Another example

In [133]:
def my_function(x):
    return x ** 2

x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
y = my_function(x)
y.sum().backward()

print("Input:", x)
print("Output:", y)
print("Gradient:", x.grad)

Input: tensor([1., 2., 3.], requires_grad=True)
Output: tensor([1., 4., 9.], grad_fn=<PowBackward0>)
Gradient: tensor([2., 4., 6.])


Another Example 2

In [134]:
import torch

# Inputs with non-uniform spacing
x_coor = torch.tensor([0.0, 0.2, 0.5, 0.8], requires_grad=True)
y_coor = torch.tensor([0.0, 0.4, 0.6], requires_grad=True)

# Create meshgrid
X, Y = torch.meshgrid(x_coor, y_coor, indexing="ij")


In [135]:
train_dataset = []
for x in x_coor:
    for y in y_coor:
        train_dataset.append((x,y,x+y))

df_train = pd.DataFrame(train_dataset, columns=['x','y','p(x,y)'])

In [136]:
X_df = df_train['x'].to_numpy()
Y_df = df_train['y'].to_numpy()

In [137]:
X_df

array([tensor(0., grad_fn=<UnbindBackward0>),
       tensor(0., grad_fn=<UnbindBackward0>),
       tensor(0., grad_fn=<UnbindBackward0>),
       tensor(0.2000, grad_fn=<UnbindBackward0>),
       tensor(0.2000, grad_fn=<UnbindBackward0>),
       tensor(0.2000, grad_fn=<UnbindBackward0>),
       tensor(0.5000, grad_fn=<UnbindBackward0>),
       tensor(0.5000, grad_fn=<UnbindBackward0>),
       tensor(0.5000, grad_fn=<UnbindBackward0>),
       tensor(0.8000, grad_fn=<UnbindBackward0>),
       tensor(0.8000, grad_fn=<UnbindBackward0>),
       tensor(0.8000, grad_fn=<UnbindBackward0>)], dtype=object)

In [138]:
Y_df

array([tensor(0., grad_fn=<UnbindBackward0>),
       tensor(0.4000, grad_fn=<UnbindBackward0>),
       tensor(0.6000, grad_fn=<UnbindBackward0>),
       tensor(0., grad_fn=<UnbindBackward0>),
       tensor(0.4000, grad_fn=<UnbindBackward0>),
       tensor(0.6000, grad_fn=<UnbindBackward0>),
       tensor(0., grad_fn=<UnbindBackward0>),
       tensor(0.4000, grad_fn=<UnbindBackward0>),
       tensor(0.6000, grad_fn=<UnbindBackward0>),
       tensor(0., grad_fn=<UnbindBackward0>),
       tensor(0.4000, grad_fn=<UnbindBackward0>),
       tensor(0.6000, grad_fn=<UnbindBackward0>)], dtype=object)

In [139]:
p_df = df_train['p(x,y)'].to_numpy().reshape(4,3)
p_df

array([[tensor(0., grad_fn=<AddBackward0>),
        tensor(0.4000, grad_fn=<AddBackward0>),
        tensor(0.6000, grad_fn=<AddBackward0>)],
       [tensor(0.2000, grad_fn=<AddBackward0>),
        tensor(0.6000, grad_fn=<AddBackward0>),
        tensor(0.8000, grad_fn=<AddBackward0>)],
       [tensor(0.5000, grad_fn=<AddBackward0>),
        tensor(0.9000, grad_fn=<AddBackward0>),
        tensor(1.1000, grad_fn=<AddBackward0>)],
       [tensor(0.8000, grad_fn=<AddBackward0>),
        tensor(1.2000, grad_fn=<AddBackward0>),
        tensor(1.4000, grad_fn=<AddBackward0>)]], dtype=object)

In [140]:
X

tensor([[0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000],
        [0.5000, 0.5000, 0.5000],
        [0.8000, 0.8000, 0.8000]], grad_fn=<ExpandBackward0>)

In [141]:
Y

tensor([[0.0000, 0.4000, 0.6000],
        [0.0000, 0.4000, 0.6000],
        [0.0000, 0.4000, 0.6000],
        [0.0000, 0.4000, 0.6000]], grad_fn=<ExpandBackward0>)

CONCAT example

In [142]:
pp = [torch.tensor([1,2,34]), torch.tensor([2,21,63])]

In [143]:
torch.concat(pp).view(2,3)

tensor([[ 1,  2, 34],
        [ 2, 21, 63]])

Own Laplacian calculation with spatial information

In [144]:
x = torch.linspace(0,1,len_x)
y = torch.linspace(0,1,len_y)

In [145]:
X, Y = torch.meshgrid(x,y, indexing='ij')

In [146]:
p = target_func(X,Y,kappa)
p = p.reshape(len_x*len_x,1)

In [147]:
p.view(len_x,len_x)

tensor([[ 0.0000, -0.0625, -0.2500, -0.5625, -1.0000],
        [ 0.0625,  0.0000, -0.1875, -0.5000, -0.9375],
        [ 0.2500,  0.1875,  0.0000, -0.3125, -0.7500],
        [ 0.5625,  0.5000,  0.3125,  0.0000, -0.4375],
        [ 1.0000,  0.9375,  0.7500,  0.4375,  0.0000]])

In [148]:
gradp_x=torch.zeros(len_x*len_x) + 10
gradp_y=torch.zeros(len_x*len_x) + 10

In [149]:
dX = 1.0/(len_x-1)

In [150]:
len_x

5

In [151]:
0.5/dX

2.0

In [152]:
def grad_x(p, len_x): 
    """Calculate the gradient of the input p in x-direction
       It assumes uniformly distributed nodes in a domain D=[0,1] x [0,1].
       Hence, len_x = len_y.
       Adjust coordinates and geometry if needed, or alternatively transform variables.
    """
    epsilon = 1e-8 #
    gradp_x=torch.zeros(len_x*len_x) + epsilon

    for i in range(1,len_x-1):
        for j in range(0,len_x):
            gradp_x[i*len_x+j] = (p[(i+1)*len_x+j] - p[(i-1)*len_x+j]) * 0.5/dX

    #One-sided gradient for @ x=0. It means i==0. 
    for j in range(0,len_x):
        gradp_x[j] = ( -p[2*len_x+j] +4*p[len_x+j] - 3*p[j]) * 0.5/dX


    #One-sided gradient for @ x=1. It means i==lenX-1. Second-order accurate
    for j in range(0,len_x):
        gradp_x[(len_x-1)*len_x+j] =  (3*p[(len_x-1)*len_x+j] - 4*p[(len_x-2)*len_x+j] + p[(len_x-3)*len_x+j]) * 0.5/dX

    return gradp_x

In [153]:
p_x= grad_x(p, len_x)

In [154]:
p_x.view(len_x,len_x)

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.5000, 0.5000, 0.5000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
        [1.5000, 1.5000, 1.5000, 1.5000, 1.5000],
        [2.0000, 2.0000, 2.0000, 2.0000, 2.0000]])

In [155]:
p_xx= grad_x(p_x, len_x)

In [156]:
p_xx.view(len_x,len_x)

tensor([[2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2.]])

In [157]:
def grad_y(p, len_x): 
    """Calculate the gradient of the input p in y-direction
       It assumes uniformly distributed nodes in a domain D=[0,1] x [0,1].
       Hence, len_x = len_y.
       Adjust coordinates and geometry if needed, or alternatively transform variables.
    """
    epsilon = 1e-8 #
    gradp_y=torch.zeros(len_x*len_x) + epsilon

    for i in range(0,len_x):
        for j in range(1,len_x-1):
            gradp_y[i*len_x+j] = (p[i*len_x+j+1] - p[i*len_x+j-1]) * 0.5/dX

    # One-sided @ y=0. Corresponds j=0.
    for i in range(0,len_x):
        gradp_y[i*len_x] = (-p[i*len_x+2] + 4*p[i*len_x+1] - 3*p[i*len_x]) * 0.5/dX

    # One-sided @ y=1. Corresponds j=lenY-1.
    for i in range(0,len_x):
        indx = (i+1)*len_x-1
        gradp_y[indx] = (3*p[indx]-4*p[indx-1] + p[indx-2]) * 0.5/dX

    
    return gradp_y

In [158]:
p_y = grad_y(p,len_y)

In [159]:
p_y.view(len_x,len_x)

tensor([[ 0.0000, -0.5000, -1.0000, -1.5000, -2.0000],
        [ 0.0000, -0.5000, -1.0000, -1.5000, -2.0000],
        [ 0.0000, -0.5000, -1.0000, -1.5000, -2.0000],
        [ 0.0000, -0.5000, -1.0000, -1.5000, -2.0000],
        [ 0.0000, -0.5000, -1.0000, -1.5000, -2.0000]])

In [160]:
p_yy = grad_y(p_y,len_y)

In [161]:
p_yy.view(len_x,len_x)

tensor([[-2., -2., -2., -2., -2.],
        [-2., -2., -2., -2., -2.],
        [-2., -2., -2., -2., -2.],
        [-2., -2., -2., -2., -2.],
        [-2., -2., -2., -2., -2.]])